In [ ]:
!pip install transformers datasets peft accelerate bitsandbytes
!pip install -U bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 16.8 MB/s eta 0:00:00


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

model_name = "OpenLLM-Ro/RoLlama3-8b-Instruct-2024-06-28"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype="float16",
    bnb_4bit_use_double_quant=True,
)

tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)
tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto",
    attn_implementation="sdpa",     # <--- IMPORTANT FIX
)

model.config.use_cache = False
model.gradient_checkpointing_enable()


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/172 [00:00<?, ?B/s]

In [ ]:
from peft import prepare_model_for_kbit_training

model.gradient_checkpointing_enable()
model.config.use_cache = False

model = prepare_model_for_kbit_training(model)


In [ ]:
from peft import LoraConfig, get_peft_model

lora_cfg = LoraConfig(
    r=8,
    lora_alpha=16,
    lora_dropout=0.05,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],
    bias="none",
    task_type="CAUSAL_LM",
)

model = get_peft_model(model, lora_cfg)


/usr/local/lib/python3.12/dist-packages/peft/mapping_func.py:73: UserWarning: You are trying to modify a model with PEFT for a second time. If you want to reload the model with a different config, make sure to call `.unload()` before.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/peft/tuners/tuners_utils.py:196: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(


In [ ]:
dataset = load_dataset("json", data_files="dataset_contracte_llama.json", split="train")
dataset = dataset.train_test_split(test_size=0.05)

train_dataset = dataset["train"]
val_dataset   = dataset["test"]

In [ ]:
def format_example(instr, inp, out):
    instr = str(instr)
    inp   = "" if inp is None else str(inp)
    out   = str(out)

    chat = [
        {"role": "system", "content": "Ești un asistent juridic."},
        {"role": "user", "content": instr if inp == "" else f"{instr}\n\n{inp}"},
        {"role": "assistant", "content": out},
    ]

    return tokenizer.apply_chat_template(chat, tokenize=False)


In [ ]:
def tokenize(batch):
    texts = []

    # iterate over examples correctly
    for instr, inp, out in zip(
        batch["instruction"],
        batch.get("input", [""] * len(batch["instruction"])),
        batch["output"]
    ):
        text = format_example(instr, inp, out)
        texts.append(text)

    tokenized = tokenizer(
        texts,
        truncation=True,
        max_length=512,
        padding="max_length",
    )

    tokenized["labels"] = tokenized["input_ids"].copy()
    return tokenized


In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./lora-finetuned",
    num_train_epochs=2,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=16,
    warmup_steps=50,
    learning_rate=2e-4,
    fp16=True,
    logging_steps=10,
    save_strategy="steps",
    save_steps=200,
    optim="paged_adamw_8bit",
)


In [ ]:
tokenized_dataset = train_dataset.map(
    tokenize,
    batched=True,
    remove_columns=train_dataset.column_names
)


Map:   0%|          | 0/114 [00:00<?, ? examples/s]

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    tokenizer=tokenizer,
)


/tmp/ipython-input-2693605849.py:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
trainer.train()


/usr/local/lib/python3.12/dist-packages/torch/_dynamo/eval_frame.py:929: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Step,Training Loss


Step,Training Loss
10,2.174800


TrainOutput(global_step=16, training_loss=2.106287181377411, metrics={'train_runtime': 496.3015, 'train_samples_per_second': 0.459, 'train_steps_per_second': 0.032, 'total_flos': 5261343175213056.0, 'train_loss': 2.106287181377411, 'epoch': 2.0})

In [ ]:
model.save_pretrained("./lora-finetuned")


Ca sa pot rula: load the adapter for the fine tuned model

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from peft import PeftModel

model_name = "OpenLLM-Ro/RoLlama3-8b-Instruct-2024-06-28"
adapter_path = "./lora-finetuned"     # your saved folder

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype="float16",
    bnb_4bit_use_double_quant=True,
)

tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)
tokenizer.pad_token = tokenizer.eos_token

# Load base model in 4-bit
base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    quantization_config=bnb_config,
    attn_implementation="sdpa",
)

# Load LoRA adapter
model = PeftModel.from_pretrained(base_model, adapter_path)
model.eval()


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

/usr/local/lib/python3.12/dist-packages/peft/peft_model.py:585: UserWarning: Found missing adapter keys while loading the checkpoint: ['base_model.model.model.layers.0.self_attn.q_proj.lora_A.default.weight', 'base_model.model.model.layers.0.self_attn.q_proj.lora_B.default.weight', 'base_model.model.model.layers.0.self_attn.k_proj.lora_A.default.weight', 'base_model.model.model.layers.0.self_attn.k_proj.lora_B.default.weight', 'base_model.model.model.layers.0.self_attn.v_proj.lora_A.default.weight', 'base_model.model.model.layers.0.self_attn.v_proj.lora_B.default.weight', 'base_model.model.model.layers.0.self_attn.o_proj.lora_A.default.weight', 'base_model.model.model.layers.0.self_attn.o_proj.lora_B.default.weight', 'base_model.model.model.layers.1.self_attn.q_proj.lora_A.default.weight', 'base_model.model.model.layers.1.self_attn.q_proj.lora_B.default.weight', 'base_model.model.model.layers.1.self_attn.k_proj.lora_A.default.weight', 'base_model.model.model.layers.1.self_attn.k_proj.l

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(128256, 4096)
        (layers): ModuleList(
          (0-31): 32 x LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): lora.

Build prompt

In [ ]:
def build_prompt(user_prompt):
    messages = [
        {"role": "system", "content": "Ești un asistent juridic."},
        {"role": "user",   "content": user_prompt}
    ]
    return tokenizer.apply_chat_template(messages, tokenize=False)


In [ ]:
import torch

def generate(prompt, max_new_tokens=350):
    text = build_prompt(prompt)
    inputs = tokenizer(text, return_tensors="pt").to(model.device)

    with torch.no_grad():
        output = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            do_sample=True,
            temperature=0.7,
            top_p=0.9,
            repetition_penalty=1.1
        )

    return tokenizer.decode(output[0], skip_special_tokens=True)


In [ ]:
response = generate("Generează un contract de vânzare-cumpărare scurt.")
print(response)


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


system

Ești un asistent juridic.user

Generează un contract de vânzare-cumpărare scurt.assistant

Aici este un document tipizat de contract de vânziuni-cumpărături care poate fi utilizat pentru o tranzacție simplă. Această formă nu include termeni și condiții specifice, cum ar fi clauze de garanție sau de revendicare a proprietății, astfel încât este important să se consulte cu un avocat înainte de utilizarea acestuia.

Vânzare-Cumpărare Contract

Acordul de încredere
De la [numele vânzătorului] („vânzător”) partea de sus a lui
și
[Name of the Buyer] („Cumpărător”)

Prezent: 
Această înțelegere prezintă acordul dintre Vânzător și Cumpărător cu privire la urmatorul produs:

1. Produsele menționate sunt descrise mai jos.
2. Prețul vânzării este [prețul de vânzare].
3. Metoda de plată acceptabilă este [plata în numerar].

În schimbul plății prețului de vânzare, Vânzătorul transferă proprietatea și toate drepturile asupra Produselor către Cumpărător. Cumpărătorul este responsabil pentru o

In [ ]:
def run_llm(messages, max_new=300, temp=0.2):
    """
    Executează Llama 3 + LoRA în mod chat și returnează textul generat.
    messages = [{"role": "user", "content": "..."}]
    """

    # 1. Construim promptul complet folosind șablonul HF al modelului
    prompt = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )

    # 2. Tokenizare
    inputs = tokenizer(
        prompt,
        return_tensors="pt"
    ).to(model.device)

    # 3. Generare
    with torch.no_grad():
        output = model.generate(
            **inputs,
            max_new_tokens=max_new,
            temperature=temp,
            top_p=0.9,
            do_sample=True
        )

    # 4. Decodare + eliminare prompt
    return tokenizer.decode(output[0], skip_special_tokens=True)


In [ ]:
import json

def critic(text, meta):
    """
    Verifică într-un singur pas atât structura (Forma) cât și conținutul (Scopul).
    """
    scop_cerut = meta.get('scop', 'Nespecificat')
    mandant_cerut = meta['mandant']['nume']

    prompt = f"""
Ești un AUDITOR NOTARIAL EXPERT.
Analizează documentul de mai jos și completează raportul de validare JSON.

DATE DE REFERINȚĂ (Ce trebuie să conțină):
1. Scopul Mandatului: "{scop_cerut}"
2. Mandant: "{mandant_cerut}"

TEXT DE VERIFICAT:
---
{text[:2000]}
--- (text trunchiat pentru analiză)

SARCINĂ DE AUDIT:
A. VERIFICARE FORMĂ:
   - Textul pare complet (are titlu, final)?
   - S-au eliminat toate placeholder-ele gen "...", "_", "[Data]"? (CRITIC!)
   - Există numele mandantului?

B. VERIFICARE SCOP:
   - Textul menționează clar acțiunea cerută ("{scop_cerut}")? (Atenție la sinonime juridice).

RĂSPUNDE DOAR CU ACEST JSON STRICT:
{{
    "forma_ok": (bool - false dacă există "..." sau lipsesc părți),
    "scop_ok": (bool - true dacă scopul e acoperit),
    "scor_calitate": (int 1-10),
    "observatii": "scurtă explicație a erorilor sau OK"
}}
"""
    # Temperatură mică pentru rigoare
    raw_res = run_llm([{"role": "user", "content": prompt}], max_new=300, temp=0.2)

    print(f"   [RAȚIONAMENT CRITIC]: {raw_res}") # Debug

    try:
        # Curățare JSON (metoda robustă)
        clean_res = raw_res.replace("json", "").replace("", "").strip()
        start = clean_res.find("{")
        end = clean_res.rfind("}") + 1

        if start != -1:
            return json.loads(clean_res[start:end])
        return None
    except:
        return None

In [ ]:
critic("Aici este un document tipizat de contract de vânziuni-cumpărături care poate fi utilizat pentru o tranzacție simplă. Această formă nu include termeni și condiții specifice, cum ar fi clauze de garanție sau de revendicare a proprietății,\n astfel încât este important să se consulte cu un avocat înainte de utilizarea acestuia. Vânzare-Cumpărare Contract\nAcordul de încredere/nDe la [numele vânzătorului] („vânzător”) partea de sus a lui /nși /n[Name of the Buyer] („Cumpărător”)\n

Prezent: Această înțelegere prezintă acordul dintre Vânzător și Cumpărător cu privire la urmatorul produs:\n
1. Produsele menționate sunt descrise mai jos.\n
2. Prețul vânzării este [prețul de vânzare].\n
3. Metoda de plată acceptabilă este [plata în numerar].\n

În schimbul plății prețului de vânzare, Vânzătorul transferă proprietatea și toate drepturile asupra Produselor către Cumpărător. Cumpărătorul este responsabil pentru obținerea oricărei documentații necesare din cauza achiziționării Produselor.\n

Semnat:\n
V")